In [1]:
'''
A. Data Engineering
'''

'''
D1. Import Naver Movie Review Raw Dataset from Github
'''
! git clone https://github.com/simonjisu/nsmc_study.git

Cloning into 'nsmc_study'...
remote: Enumerating objects: 107, done.
remote: Total 107 (delta 0), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (107/107), 164.79 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Checking out files: 100% (26/26), done.


In [2]:
'''
D2. Install torchtext Libraries
'''
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 34.5 MB/s 
     |████████████████████████████████| 831.4 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [25]:
'''
D3. Import Libraries for Data Engineering
'''

import torch
from torchtext.legacy import data
import torch.nn.functional as F

import re
import numpy as np
from sklearn.model_selection import train_test_split    

# import pandas as pd
import torch.optim as optim

import random
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
'''
D4. Tokenizer Install & import
''' 
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

from konlpy.tag import Mecab

tokenizer = Mecab()

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 29.57 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 88.7 MB/s 
     |████████████████████████████████| 465 kB 72.4 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-01 06:49:33--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (

In [5]:
'''
D5. Load and modifiy to pandas dataframe
'''
import pandas as pd

train = pd.read_csv("/content/nsmc_study/data/ratings_train.txt", sep='\t')
test = pd.read_csv("/content/nsmc_study/data/ratings_test.txt", sep='\t')

train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

print(train.shape)
print(test.shape)

train_data = train.dropna() #말뭉치에서 nan 값을 제거함
test_data  = test.dropna()

(150000, 2)
(50000, 2)


In [6]:
'''
D6. Preprocess and build list
'''
def preprocess_func(sentence):
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    sentence = sentence.strip()
    return sentence

train_data['document'] = train_data['document'].apply(preprocess_func)
test_data['document']  = test_data['document'].apply(preprocess_func)

<ipython-input-6-2bdbc54489ce>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['document'] = train_data['document'].apply(preprocess_func)
<ipython-input-6-2bdbc54489ce>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['document']  = test_data['document'].apply(preprocess_func)


In [7]:
'''
D7. Split Data
'''
train_data, valid_data = train_test_split(train_data, test_size=0.3, random_state=32)

print(len(train_data))
print(len(valid_data))
print(len(test_data))
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

104996
44999
49997
(104996, 2)
(44999, 2)
(49997, 2)


In [8]:
'''
D8. Tokenizer define
'''

TEXT = data.Field(sequential=True, use_vocab=True, tokenize=tokenizer.morphs, lower=False, batch_first=True, fix_length=50)

LABEL = data.LabelField(dtype = torch.float)

In [9]:
'''
D9. Tokenizer test
# PASS
'''

'''
D10. Pad sequence
# PASS
'''

'\nD10. Pad sequence\n# PASS\n'

In [10]:
'''
D11. Convert dataset
'''
def convert_dataset(input_data, text, label):
    list_of_example = [data.Example.fromlist(row.tolist(), fields=[('text', text), ('label', label)])  for _, row in input_data.iterrows()]
    dataset = data.Dataset(examples=list_of_example, fields=[('text', text), ('label', label)])
    return dataset

train_data = convert_dataset(train_data, TEXT, LABEL)
valid_data = convert_dataset(valid_data, TEXT, LABEL)
test_data  = convert_dataset(test_data, TEXT, LABEL)

print(f'Number of training examples   : {len(train_data)}')
print(f'Number of validation examples : {len(valid_data)}')
print(f'Number of testing examples    : {len(test_data)}')

Number of training examples   : 104996
Number of validation examples : 44999
Number of testing examples    : 49997


In [11]:
'''
D12. Build vocaburary
'''
MAX_VOCAB_SIZE = 20000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)

LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary : {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

print(TEXT.vocab.freqs.most_common(20))

print(TEXT.vocab.itos[:10])

print(LABEL.vocab.stoi)

Unique tokens in TEXT vocabulary : 20002
Unique tokens in LABEL vocabulary: 2
[('.', 169097), ('이', 51416), ('는', 46909), ('영화', 40488), ('다', 39051), ('고', 33098), ('하', 31032), ('도', 23822), ('의', 23626), ('가', 23389), ('!', 22218), ('은', 21704), ('에', 21588), ('을', 20807), ('보', 18133), ('한', 17482), (',', 15841), ('게', 15462), ('들', 15038), ('지', 13373)]
['<unk>', '<pad>', '.', '이', '는', '영화', '다', '고', '하', '도']
defaultdict(None, {0: 0, 1: 1})


In [12]:
'''
D13. Dataload with Iterator
# PASS
'''

'''
D14. Data type define
# PASS
'''

'\nD14. Data type define\n# PASS\n'

In [13]:
'''
D15. Dataload with Iterator
'''
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.Iterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort = False,
    device = device)

print('Number of minibatch for training dataset   : {}'.format(len(train_iterator)))
print('Number of minibatch for validation dataset : {}'.format(len(valid_iterator)))
print('Number of minibatch for testing dataset    : {}'.format(len(test_iterator)))

Number of minibatch for training dataset   : 1641
Number of minibatch for validation dataset : 704
Number of minibatch for testing dataset    : 782


In [14]:
'''
B. Model Engineering
'''

'''
M1. Import Libraries for Model Engineering
'''
import torch.nn as nn

In [15]:
'''
M2. Set Hyperparameters
'''
embedding_dim = 256
hidden_units = 128
EPOCHS = 20
learning_rate = 5e-4

In [16]:
'''
M4. Build NN model
'''
class LSTM(nn.Module):
    def __init__(self, vocab_size, hidden_dim, output_dim, embedding_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, _ = self.rnn(embedded)
        output = self.linear(output[:, -1, :])
        return output
  
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

model = LSTM(len(TEXT.vocab), 128, len(LABEL.vocab)-1, 300, 0.2)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,220,889 trainable parameters


/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [17]:
'''
M5. Optimizer
'''
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
'''
M6. Define Loss Function
'''
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [19]:
'''
M7. Define Accuracy Function
'''
def binary_accuracy(preds, target):
    '''
    from https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
    '''
    # round predictions to the closest integer (0 or 1)
    rounded_preds = torch.round(torch.sigmoid(preds))

    #convert into float for division
    correct = (rounded_preds == target).float()

    # rounded_preds = [ 1   0   0   1   1   1   0   1   1   1]
    # targets       = [ 1   0   1   1   1   1   0   1   1   0]
    # correct       = [1.0 1.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0 0.0]
    acc = correct.sum() / len(correct)
    return acc

In [20]:
'''
M8. Define Training Function
'''
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        # We initialize the gradient to 0 for every batch.
        optimizer.zero_grad()

        # batch of sentences인 batch.text를 model에 입력
        predictions = model(batch.text).squeeze(1)
        
        # Calculate the loss value by comparing the prediction result with batch.label 
        loss = criterion(predictions, batch.label)

        # Accuracy calculation
        acc = binary_accuracy(predictions, batch.label)
        
        # Backpropagation using backward()
        loss.backward()

        # Update the parameters using the optimization algorithm
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
'''
M9. Define Validation / Test Function
'''
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    # "evaluation mode" : turn off "dropout" or "batch nomalizaation"
    model.eval()

    # Use less memory and speed up computation by preventing gradients from being computed in pytorch
    with torch.no_grad():
    
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

best_valid_loss = float('inf')

'''
M10. Episode / each step Process
'''
for epoch in range(EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.585 | Train Acc: 65.18%
	 Val. Loss: 0.425 |  Val. Acc: 81.75%
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.373 | Train Acc: 83.57%
	 Val. Loss: 0.353 |  Val. Acc: 84.95%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.316 | Train Acc: 86.51%
	 Val. Loss: 0.337 |  Val. Acc: 85.68%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.280 | Train Acc: 88.39%
	 Val. Loss: 0.332 |  Val. Acc: 86.19%
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.251 | Train Acc: 89.68%
	 Val. Loss: 0.336 |  Val. Acc: 86.34%
Epoch: 06 | Epoch Time: 0m 13s
	Train Loss: 0.226 | Train Acc: 90.94%
	 Val. Loss: 0.352 |  Val. Acc: 86.37%
Epoch: 07 | Epoch Time: 0m 12s
	Train Loss: 0.205 | Train Acc: 91.81%
	 Val. Loss: 0.354 |  Val. Acc: 86.11%
Epoch: 08 | Epoch Time: 0m 13s
	Train Loss: 0.186 | Train Acc: 92.80%
	 Val. Loss: 0.355 |  Val. Acc: 86.17%
Epoch: 09 | Epoch Time: 0m 13s
	Train Loss: 0.168 | Train Acc: 93.51%
	 Val. Loss: 0.372 |  Val. Acc: 86.02%
Epoch: 10 | Epoch T

In [23]:
'''
M11. Assess model performance (Test step)
'''
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

import torch
model.load_state_dict(torch.load('tut4-model.pt'))

Test Loss: 0.338 | Test Acc: 86.01%


<All keys matched successfully>

In [26]:
'''
M12. [Opt] Training result test for Code Engineering
'''
def predict(model,sentence):
    model.eval()
    with torch.no_grad():
        sent = tokenizer.morphs(sentence)
        sent = torch.tensor([TEXT.vocab.stoi[i] for i in sent])
        sent = F.pad(sent,pad = (1,50-len(sent)-1),value = 1)
        sent = sent.unsqueeze(dim = 0) #for batch
        output = torch.sigmoid(model(sent))

        return output.item()
        
examples = [
  "재미 있어요! 꼭 보세요!",
  "추천하기 어렵네요. 그다지 재미는 없는듯!",
  "너무 신나요. 인생 역작입니다. 강추"
]

model = model.to('cpu')
for idx in range(len(examples)) :

    sentence = examples[idx]
    pred = predict(model,sentence)
    print("\n",sentence)
    if pred >= 0.5 :
        print(f">>>긍정 리뷰입니다. ({pred : .2f})")
    else:
        print(f">>>부정 리뷰입니다.({pred : .2f})")


 재미 있어요! 꼭 보세요!
>>>긍정 리뷰입니다. ( 0.99)

 추천하기 어렵네요. 그다지 재미는 없는듯!
>>>부정 리뷰입니다.( 0.04)

 너무 신나요. 인생 역작입니다. 강추
>>>긍정 리뷰입니다. ( 0.99)
